# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7d14a57970>
├── 'a' --> tensor([[ 1.2364, -1.1034, -0.6518],
│                   [-0.8774,  0.0566, -1.0533]])
└── 'x' --> <FastTreeValue 0x7f7d14a57340>
    └── 'c' --> tensor([[-0.0288, -2.4260, -0.4094,  1.3185],
                        [-0.0968, -0.7687, -0.7208, -0.0608],
                        [ 0.2771,  0.9496,  0.5076, -0.4765]])

In [4]:
t.a

tensor([[ 1.2364, -1.1034, -0.6518],
        [-0.8774,  0.0566, -1.0533]])

In [5]:
%timeit t.a

63.8 ns ± 0.0801 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7d14a57970>
├── 'a' --> tensor([[-0.6656, -0.0113,  0.3428],
│                   [ 1.2313,  0.1045,  0.0771]])
└── 'x' --> <FastTreeValue 0x7f7d14a57340>
    └── 'c' --> tensor([[-0.0288, -2.4260, -0.4094,  1.3185],
                        [-0.0968, -0.7687, -0.7208, -0.0608],
                        [ 0.2771,  0.9496,  0.5076, -0.4765]])

In [7]:
%timeit t.a = new_value

66.1 ns ± 0.0997 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2364, -1.1034, -0.6518],
               [-0.8774,  0.0566, -1.0533]]),
    x: Batch(
           c: tensor([[-0.0288, -2.4260, -0.4094,  1.3185],
                      [-0.0968, -0.7687, -0.7208, -0.0608],
                      [ 0.2771,  0.9496,  0.5076, -0.4765]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2364, -1.1034, -0.6518],
        [-0.8774,  0.0566, -1.0533]])

In [11]:
%timeit b.a

62.1 ns ± 0.0171 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.5364,  0.3528,  0.2772],
               [-0.6556, -1.3578, -0.0943]]),
    x: Batch(
           c: tensor([[-0.0288, -2.4260, -0.4094,  1.3185],
                      [-0.0968, -0.7687, -0.7208, -0.0608],
                      [ 0.2771,  0.9496,  0.5076, -0.4765]]),
       ),
)

In [13]:
%timeit b.a = new_value

503 ns ± 1.04 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

829 ns ± 4.58 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 20.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 438 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7c6a723550>
├── 'a' --> tensor([[[ 1.2364, -1.1034, -0.6518],
│                    [-0.8774,  0.0566, -1.0533]],
│           
│                   [[ 1.2364, -1.1034, -0.6518],
│                    [-0.8774,  0.0566, -1.0533]],
│           
│                   [[ 1.2364, -1.1034, -0.6518],
│                    [-0.8774,  0.0566, -1.0533]],
│           
│                   [[ 1.2364, -1.1034, -0.6518],
│                    [-0.8774,  0.0566, -1.0533]],
│           
│                   [[ 1.2364, -1.1034, -0.6518],
│                    [-0.8774,  0.0566, -1.0533]],
│           
│                   [[ 1.2364, -1.1034, -0.6518],
│                    [-0.8774,  0.0566, -1.0533]],
│           
│                   [[ 1.2364, -1.1034, -0.6518],
│                    [-0.8774,  0.0566, -1.0533]],
│           
│                   [[ 1.2364, -1.1034, -0.6518],
│                    [-0.8774,  0.0566, -1.0533]]])
└── 'x' --> <FastTreeValue 0x7f7c6a723430>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 68.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7c6a740d30>
├── 'a' --> tensor([[ 1.2364, -1.1034, -0.6518],
│                   [-0.8774,  0.0566, -1.0533],
│                   [ 1.2364, -1.1034, -0.6518],
│                   [-0.8774,  0.0566, -1.0533],
│                   [ 1.2364, -1.1034, -0.6518],
│                   [-0.8774,  0.0566, -1.0533],
│                   [ 1.2364, -1.1034, -0.6518],
│                   [-0.8774,  0.0566, -1.0533],
│                   [ 1.2364, -1.1034, -0.6518],
│                   [-0.8774,  0.0566, -1.0533],
│                   [ 1.2364, -1.1034, -0.6518],
│                   [-0.8774,  0.0566, -1.0533],
│                   [ 1.2364, -1.1034, -0.6518],
│                   [-0.8774,  0.0566, -1.0533],
│                   [ 1.2364, -1.1034, -0.6518],
│                   [-0.8774,  0.0566, -1.0533]])
└── 'x' --> <FastTreeValue 0x7f7c6a78cfd0>
    └── 'c' --> tensor([[-0.0288, -2.4260, -0.4094,  1.3185],
                        [-0.0968, -0.7687, -0.7208, -0.0608],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 53 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.5 µs ± 83.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.0288, -2.4260, -0.4094,  1.3185],
                       [-0.0968, -0.7687, -0.7208, -0.0608],
                       [ 0.2771,  0.9496,  0.5076, -0.4765]],
              
                      [[-0.0288, -2.4260, -0.4094,  1.3185],
                       [-0.0968, -0.7687, -0.7208, -0.0608],
                       [ 0.2771,  0.9496,  0.5076, -0.4765]],
              
                      [[-0.0288, -2.4260, -0.4094,  1.3185],
                       [-0.0968, -0.7687, -0.7208, -0.0608],
                       [ 0.2771,  0.9496,  0.5076, -0.4765]],
              
                      [[-0.0288, -2.4260, -0.4094,  1.3185],
                       [-0.0968, -0.7687, -0.7208, -0.0608],
                       [ 0.2771,  0.9496,  0.5076, -0.4765]],
              
                      [[-0.0288, -2.4260, -0.4094,  1.3185],
                       [-0.0968, -0.7687, -0.7208, -0.0608],
                       [ 0.2771,  0.9496,  0.5076, -0.4765]],

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 69.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.0288, -2.4260, -0.4094,  1.3185],
                      [-0.0968, -0.7687, -0.7208, -0.0608],
                      [ 0.2771,  0.9496,  0.5076, -0.4765],
                      [-0.0288, -2.4260, -0.4094,  1.3185],
                      [-0.0968, -0.7687, -0.7208, -0.0608],
                      [ 0.2771,  0.9496,  0.5076, -0.4765],
                      [-0.0288, -2.4260, -0.4094,  1.3185],
                      [-0.0968, -0.7687, -0.7208, -0.0608],
                      [ 0.2771,  0.9496,  0.5076, -0.4765],
                      [-0.0288, -2.4260, -0.4094,  1.3185],
                      [-0.0968, -0.7687, -0.7208, -0.0608],
                      [ 0.2771,  0.9496,  0.5076, -0.4765],
                      [-0.0288, -2.4260, -0.4094,  1.3185],
                      [-0.0968, -0.7687, -0.7208, -0.0608],
                      [ 0.2771,  0.9496,  0.5076, -0.4765],
                      [-0.0288, -2.4260, -0.4094,  1.3185],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 412 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

330 µs ± 3.58 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
